In [1]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import cv2
import os
from tensorflow.contrib.learn.python.learn.estimators import kmeans
from tensorflow.python.framework import constant_op

In [2]:
train_set = np.load('/data/gt_proposals/embeddings_baseline_train.npy')
val_set = np.load('/data/gt_proposals/embeddings_baseline.npy')

In [3]:
clusterer = kmeans.KMeansClustering(
      num_clusters=2,
      use_mini_batch=False)
x = train_set
labels = x[:,0:1]
x = x[:,1:]
print train_set.shape
print x.shape
print labels.shape

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9dac500690>, '_model_dir': '/tmp/tmp9KoEAn', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
(26241, 2049)
(26241, 2048)
(26241, 1)


In [4]:
def _input_fn(tensor, num_epochs=None):
    """Constructs an input_fn for KMeansClustering estimator.

    Args:
      tensor: A 2D numpy.ndarray of data points.
      num_epochs: A positive integer (optional).  If specified, limits the
        number of steps the output tensor may be evaluated.

    Returns:
      An input_fn.
    """
    def _constant_input_fn():
          return (
          tf.train.limit_epochs(tf.constant(tensor), num_epochs=num_epochs),
          None)
    return _constant_input_fn

clusterer.fit(
      input_fn=_input_fn(x.astype(np.float32)), steps=10)

Instructions for updating:
Please switch to tf.train.get_global_step
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp9KoEAn/model.ckpt.
INFO:tensorflow:loss = 5.42514e+06, step = 1
INFO:tensorflow:Saving checkpoints for 10 into /tmp/tmp9KoEAn/model.ckpt.
INFO:tensorflow:Loss for final step: 3.19265e+06.


KMeansClustering(params={'mini_batch_steps_per_iteration': 1, 'random_seed': 0, 'training_initial_clusters': 'random', 'use_mini_batch': False, 'distance_metric': 'squared_euclidean', 'num_clusters': 2, 'kmeans_plus_plus_num_retries': 2, 'relative_tolerance': None})

In [5]:
x_val = val_set
labels_val = x_val[:,0:1]
x_val = x_val[:,1:]
predictions = np.array(list(clusterer.predict_cluster_idx(
        input_fn=_input_fn(x_val.astype(np.float32), num_epochs=1))))

INFO:tensorflow:Restoring parameters from /tmp/tmp9KoEAn/model.ckpt-10


In [6]:
print labels_val.shape
print predictions.shape

(5257, 1)
(5257,)


In [7]:
cluster_counts = dict()
for l,p in zip(labels_val.flatten(),predictions):
    if l not in cluster_counts:
        cluster_counts[l] = [0,0,0]
    cluster_counts[l][p] += 1

In [8]:
print cluster_counts

{11.0: [1737, 112, 0], 12.0: [300, 45, 0], 13.0: [198, 1960, 0], 14.0: [5, 54, 0], 15.0: [10, 70, 0], 16.0: [8, 11, 0], 17.0: [21, 10, 0], 18.0: [644, 72, 0]}


In [9]:
tf.reset_default_graph()